In [21]:
import time
import pandas as pd
from census import Census
import altair as alt
import numpy as np

# Census API access
api_key = "639f2aedf7c17b164527591258cda00b25249b4b"
c = Census(key=api_key)

In [43]:
# Rent: Table B25058
rent_variables = {
    'B25058_001E': 'median_rent',
    'B25058_001M': 'median_rent_moe'
}
rent_columns_out = [
    'median_rent', 'median_rent_moe'
]
renter_units = {
    "2014_tracts": 5456.00,
    "2019_tracts": 5482.00,
    "2014_city": 18414.02,
    "2019_city": 18606.01
}

In [49]:
def adjust_inflation(in_df, year):
    ## Adjust median rent for inflation
    inflation_rate = {
        2014: 1.32,
        2017: 1.27,
        2019: 1.23,
        2022: 1.10
    }
    df = pd.DataFrame()

    for c in rent_variables.values():
        df[c] = in_df[c] * inflation_rate[year]

    return df

In [78]:
def combine_tracts(in_df):
    '''

    Outputs:
    A modified version of in_df with census tracts combined
    '''
    
    # Start by making a copy of in_df, so we don't destroy the original data.
    # df is also a nice short name we can use throughout this function
    df_copy = in_df.copy()
    df = pd.DataFrame()

    ### CLEAN UNUSUAL MOES
    df_copy = df_copy.replace(-555555555.0, 0)

    for c in rent_variables.values():
        if "moe" not in c:
            # sum the totals: aggregate rent
            df.at[0, f'{c}_agg'] = df_copy[c].sum()
        else:
            # sum of squares the moes
            df.at[0, f'{c}_agg'] = np.sqrt((df_copy[c]**2).sum())
    
    return df

In [97]:
def get_rent_precombo(year_in, place_num):
    # for getting mode for census tracts before they are combined
    df = pd.DataFrame(
        c.acs5.get(
            list(rent_variables.keys()),
            {'for': place_num, 'in': 'state:06 county:013'},
            year=year_in
        )
    )
    df = df.rename(columns=rent_variables)
    df = df.drop(columns=["state", "county", "tract"])
    
    df_adjusted = adjust_inflation(df, year_in)
    
    return df_adjusted

In [83]:
def get_city_df(year_in):
    # City
    df = pd.DataFrame(
        c.acs5.get(
            list(rent_variables.keys()),
            {'for': 'place:60620', 'in': 'state:06'},
            year=year_in
        )
    )
    df = df.rename(columns=rent_variables)

    df_adjusted = adjust_inflation(df, year_in)
    
    return df_adjusted

In [98]:
# for 5yr ACS 2014 and 2019
# Get ACS Table B08141 for select tracts around the BART station
# county:013
# city: 60620
c_tracts = 'tract:375000, 376000, 377000, 374000, 381000'

df_tracts_2014 = get_rent_precombo(2014, c_tracts)
df_tracts_2019 = get_rent_precombo(2019, c_tracts)

In [79]:
df_comb_2014 = combine_tracts(df_tracts_2014)
# df_tracts_out_2014 = df_comb_2014[rent_columns_out]
# df_tracts_out_2014.insert(0, "year", 2014)
# df_comb_2019 = combine_tracts(df_tracts_2019)
# df_tracts_out_2019 = df_comb_2019[rent_columns_out]
# df_tracts_out_2019.insert(0, "year", 2019)

In [99]:
df_tracts_2019

,median_rent,median_rent_moe
0,1686.33,134.07
1,1412.04,140.22
2,1368.99,188.19
3,1452.63,295.20
4,1392.36,129.15


In [100]:
# for 5yr ACS 2017 and 2022
df_tracts_2017 = get_rent_precombo(2017, c_tracts)
df_tracts_2022 = get_rent_precombo(2022, c_tracts)

In [102]:
df_tracts_2022

,median_rent,median_rent_moe
0,1527.9,231.0
1,1426.7,218.9
2,1383.8,227.7
3,1260.6,148.5
4,1301.3,211.2


In [57]:
df_comb_2017 = combine_tracts(df_tracts_2017)
df_tracts_out_2017 = df_comb_2017[rent_columns_out]
df_tracts_out_2017.insert(0, "year", 2017)
df_comb_2022 = combine_tracts(df_tracts_2022)
df_tracts_out_2022 = df_comb_2022[rent_columns_out]
df_tracts_out_2022.insert(0, "year", 2022)

In [54]:
df_tracts_out_2014

,year,median_rent,median_rent_moe
0,2014,6487.8,304.857136


In [84]:
# for 5yr ACS 2014 and 2019
# Get ACS Table B08141 in Richmond City
df_city_2014 = get_city_df(2014)
df_city_2019 = get_city_df(2019)

In [85]:
df_city_out_2014 = df_city_2014[rent_columns_out]
df_city_out_2014.insert(0, "year", 2014)
df_city_out_2019 = df_city_2019[rent_columns_out]
df_city_out_2019.insert(0, "year", 2019)

In [88]:
# for 5yr ACS 2017 and 2022
# Get ACS Table B08141 in Richmond City
df_city_2017 = get_city_df(2017)
df_city_2022 = get_city_df(2022)

In [89]:
df_city_out_2017 = df_city_2017[rent_columns_out]
df_city_out_2017.insert(0, "year", 2017)
df_city_out_2022 = df_city_2022[rent_columns_out]
df_city_out_2022.insert(0, "year", 2022)

In [62]:
df_city_out_2014

,year,median_rent,median_rent_moe
0,2014,1450.68,34.32


In [90]:
#export settings, for a combined csv
#df_tracts_out = pd.concat([df_tracts_out_2017, df_tracts_out_2022])
df_city_out = pd.concat([df_city_out_2017, df_city_out_2022])

In [64]:
df_tracts_out

,year,median_rent,median_rent_moe
0,2017,6816.09,354.991694
0,2022,6900.3,468.810399


In [91]:
df_city_out

,year,median_rent,median_rent_moe
0,2017,1496.06,39.37
0,2022,1823.80,89.10
